# use python 2

In [ ]:
if __name__ == '__main__':


    # Load the training and test data
    # Reads a dataset in the CoNLL format from a file
    # The format is one token per line:
    # [word] [POS] [syntactic chunk] *potential junk column* [NER tag]
    # One blank line appears after each sentence
    train = read_data("data/eng.train")
    dev = read_data("data/eng.testa")
    # Here's a few sentences...
    # print "Examples of sentences:"
    # print str(dev[1])
    # print str(dev[3])
    # print str(dev[5])
    system_to_run = 'CRF'
    # Set to True when you're ready to run your CRF on the test set to produce the final output
    run_on_test = True
    # Train our model
    if system_to_run == "BAD":
        bad_model = train_bad_ner_model(train)
        dev_decoded = [bad_model.decode(test_ex) for test_ex in dev]
    elif system_to_run == "HMM":
        hmm_model = train_hmm_model(train)
        dev_decoded = [hmm_model.decode(test_ex) for test_ex in dev]
    elif system_to_run == "CRF":
        crf_model = train_crf_model(train, 1, 0.1, weights_file='', output_weights='')
        dev_decoded = [crf_model.decode(test_ex) for test_ex in dev]
        if run_on_test:
            test = read_data("data/eng.testb.blind")
            test_decoded = [crf_model.decode(test_ex) for test_ex in test]
            print_output(test_decoded, "eng.testb.out")
    else:
        raise Exception("Pass in either BAD, HMM, or CRF to run the appropriate system")
    # Print the evaluation statistics
    print_evaluation(dev, dev_decoded)

Extracting features
Ex 0/14934
Ex 500/14934
Ex 1000/14934
Ex 1500/14934
Ex 2000/14934
Ex 2500/14934
Ex 3000/14934
Ex 3500/14934
Ex 4000/14934
Ex 4500/14934
Ex 5000/14934
Ex 5500/14934
Ex 6000/14934
Ex 6500/14934
Ex 7000/14934
Ex 7500/14934
Ex 8000/14934
Ex 8500/14934
Ex 9000/14934
Ex 9500/14934
Ex 10000/14934
Ex 10500/14934
Ex 11000/14934
Ex 11500/14934
Ex 12000/14934
Ex 12500/14934
Ex 13000/14934
Ex 13500/14934
Ex 14000/14934
Ex 14500/14934
Initital Statistics
Epoch 1
Training on 0
Training on 500
Training on 1000
Training on 1500
Training on 2000
Training on 2500
Training on 3000
Training on 3500
Training on 4000
Training on 4500
Training on 5000
Training on 5500
Training on 6000
Training on 6500
Training on 7000
Training on 7500
Training on 8000
Training on 8500
Training on 9000
Training on 9500
Training on 10000
Training on 10500
Training on 11000
Training on 11500
Training on 12000
Training on 12500
Training on 13000
Training on 13500
Training on 14000
Training on 14500
Labeled F1

In [ ]:
# Reads a dataset in the CoNLL format from a file
# The format is one token per line:
# [word] [POS] [syntactic chunk] *potential junk column* [NER tag]
# One blank line appears after each sentence
def read_data(file):
    f = open(file)
    sentences = []
    curr_tokens = []
    curr_bio_tags = []
    for line in f:
        stripped = line.strip()
        if stripped != "":
            fields = stripped.split(" ")
            if len(fields) == 4 or len(fields) == 5:
                # TODO: Modify this line to remember POS tags (fields[1]) or chunks (fields[2]) if desired
                curr_tokens.append(Token(fields[0], fields[1], fields[2]))
                # N.B. fields[-1] because there are weird extra fields in .train and .testa
                curr_bio_tags.append(fields[-1])
        elif stripped == "" and len(curr_tokens) > 0:
            sentences.append(LabeledSentence(curr_tokens, chunks_from_bio_tag_seq(curr_bio_tags)))
            curr_tokens = []
            curr_bio_tags = []
    return sentences

In [ ]:
# Abstraction to bundle words with POS and chunks for featurization
class Token:
    def __init__(self, word, pos, chunk):
        self.word = word
        self.pos = pos
        self.chunk = chunk

    def __repr__(self):
        return self.word
    
# Thin wrapper over a sequence of Tokens representing a sentence and an optional set of chunks
# representation NER labels, which are also stored as BIO tags
class LabeledSentence:
    def __init__(self, tokens, chunks=None):
        self.tokens = tokens
        self.chunks = chunks
        if chunks is None:
            self.bio_tags = None
        else:
            self.bio_tags = bio_tags_from_chunks(
                self.chunks, 
                len(self.tokens))

    def __repr__(self):
        return repr([repr(tok) for tok in self.tokens]) + "\n" + repr([repr(chunk) for chunk in self.chunks])

    def __len__(self):
        return len(self.tokens)

    def get_bio_tags(self):
        return self.bio_tags

# Convert BIO tags to (start, end, label) chunk representations
# (start, end) are semi-inclusive, meaning that in the sentence
# He met Barack Obama yesterday
# Barack Obama has the span (2, 4)
# N.B. this method only works because chunks are non-overlapping in this data
def chunks_from_bio_tag_seq(bio_tags):
    chunks = []
    curr_tok_start = -1
    curr_tok_label = ""
    for idx, tag in enumerate(bio_tags):
        if isB(tag):
            label = get_tag_label(tag)
            if curr_tok_label != "":
                chunks.append(Chunk(curr_tok_start, idx, curr_tok_label))
            curr_tok_label = label
            curr_tok_start = idx
        elif isI(tag):
            label = get_tag_label(tag)
            # if label != curr_tok_label:
                # print "WARNING: invalid tag sequence (I after O); ignoring the I: " + repr(bio_tags)
        else: # isO(tag):
            if curr_tok_label != "":
                chunks.append(Chunk(curr_tok_start, idx, curr_tok_label))
            curr_tok_label = ""
            curr_tok_start = -1
    return chunks

In [ ]:
# Converts a chunk representation back to BIO tags
def bio_tags_from_chunks(chunks, sent_len):
    tags = []
    for i in xrange(0, sent_len):
        matching_chunks = filter(lambda chunk: chunk.start_idx <= i and i < chunk.end_idx, chunks)
        if len(matching_chunks) > 0:
            if i == matching_chunks[0].start_idx:
                tags.append("B-" + matching_chunks[0].label)
            else:
                tags.append("I-" + matching_chunks[0].label)
        else:
            tags.append("O")
    return tags

def isB(ner_tag):
    return ner_tag.startswith("B")


def isI(ner_tag):
    return ner_tag.startswith("I")


def isO(ner_tag):
    return ner_tag == "O"

# Thin wrapper around a start and end index coupled with a label, representing,
# e.g., a chunk PER over the span (3,5). Indices are semi-inclusive, so (3,5)
# contains tokens 3 and 4 (0-based indexing).
class Chunk:
    def __init__(self, start_idx, end_idx, label):
        self.start_idx = start_idx
        self.end_idx = end_idx
        self.label = label

    def __repr__(self):
        return "(" + repr(self.start_idx) + ", " + repr(self.end_idx) + ", " + self.label + ")"

    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.start_idx == other.start_idx and self.end_idx == other.end_idx and self.label == other.label
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)

    def __hash__(self):
        return hash(self.start_idx) + hash(self.end_idx) + hash(self.label)

In [ ]:
def train_crf_model(sentences, epochs, lr, weights_file="", output_weights=""):
    tag_indexer = Indexer()
    for sentence in sentences:
        for tag in sentence.get_bio_tags():
            tag_indexer.get_index(tag)
    transition_mat = np.ones((len(tag_indexer), len(tag_indexer)))
    for tag_idxa in range(0, len(tag_indexer)):
        for tag_idxb in range(0, len(tag_indexer)):
            tag_a = tag_indexer.get_object(tag_idxa)
            tag_b = tag_indexer.get_object(tag_idxb)
            if isI(tag_b) and (get_tag_label(tag_b) != get_tag_label(tag_a)):
                transition_mat[tag_idxa][tag_idxb] = 0
    
    
    print "Extracting features"
    
    
    feature_indexer = Indexer()
    # 4-d list indexed by sentence index, word index, tag index, feature index
    feature_cache = [
        [
            [
                [] for k in xrange(0, len(tag_indexer))
            ] for j in xrange(0, len(sentences[i]))
        ] for i in xrange(0, len(sentences))
    ]
    
    
    for sentence_idx in xrange(0, len(sentences)):
        if sentence_idx % 500 == 0:
            print("Ex " + repr(sentence_idx) + "/" + repr(len(sentences)))
        for word_idx in xrange(0, len(sentences[sentence_idx])):
            for tag_idx in xrange(0, len(tag_indexer)):
                feature_cache[sentence_idx][word_idx][tag_idx] = extract_emission_features(
                    sentences[sentence_idx], 
                    word_idx, tag_indexer.get_object(tag_idx), 
                    feature_indexer, 
                    add_to_indexer=True)
                
    
    feature_weights = np.random.rand((len(feature_indexer)))
    if weights_file != "":
        feature_weights = np.load(weights_file)

    print("Initital Statistics")
    model = CrfNerModel(tag_indexer, feature_indexer, feature_weights)
    # TODO : currently using only emission features, also extend to transition features if possible
    batch_size = 1
    # training loop
    for epoch in range(0, epochs):
        print("Epoch %d" % (epoch+1))
        gradient = Counter()
        for sentence_idx in range(0, len(sentences)):
            if sentence_idx%500 == 0:
                print('Training on ' + repr(sentence_idx))
            log_marginal_probs = compute_log_marginals(sentences[sentence_idx], 
                                                       tag_indexer, 
                                                       feature_cache[sentence_idx], 
                                                       model.feature_weights)

            for word_idx in range(0, len(sentences[sentence_idx])):
                for tag_idx in range(0, len(tag_indexer)):
                    gradient.increment_all(
                        feature_cache[sentence_idx][word_idx][tag_idx], 
                        - np.exp(log_marginal_probs[word_idx][tag_idx])
                    )
                
                gold_tag = sentences[sentence_idx].get_bio_tags()[word_idx]
                gold_tag_idx = tag_indexer.index_of(gold_tag)
                gradient.increment_all(feature_cache[sentence_idx][word_idx][gold_tag_idx], 1.0)
                
            if (sentence_idx+1) % batch_size == 0:
                for weight_idx in gradient.keys():
                    model.feature_weights[weight_idx] += (lr * gradient.get_count(weight_idx))/batch_size
                gradient = Counter()
    np.save(output_weights, model.feature_weights)
    return model

In [ ]:
class Indexer(object):
    def __init__(self):
        self.objs_to_ints = {}
        self.ints_to_objs = {}

    def __repr__(self):
        return str([str(self.get_object(i)) for i in xrange(0, len(self))])

    def __len__(self):
        return len(self.objs_to_ints)

    def get_object(self, index):
        if (index not in self.ints_to_objs):
            return None
        else:
            return self.ints_to_objs[index]

    def contains(self, object):
        return self.index_of(object) != -1

    # Returns -1 if the object isn't present, index otherwise
    def index_of(self, object):
        if (object not in self.objs_to_ints):
            return -1
        else:
            return self.objs_to_ints[object]

    # Adds the object to the index if it isn't present, always returns a nonnegative index
    def get_index(self, object, add=True):
        if not add:
            return self.index_of(object)
        if (object not in self.objs_to_ints):
            new_idx = len(self.objs_to_ints)
            self.objs_to_ints[object] = new_idx
            self.ints_to_objs[new_idx] = object
        return self.objs_to_ints[object]
    
# Gets the label component of the NER tag: e.g., returns PER for B-PER
def get_tag_label(ner_tag):
    if len(ner_tag) > 2:
        return ner_tag[2:]
    else:
        return None
    
def extract_emission_features(sentence, word_index, tag, feature_indexer, add_to_indexer):
    feats = []
    curr_word = sentence.tokens[word_index].word
    # Lexical and POS features on this word, the previous, and the next (Word-1, Word0, Word1)
    for idx_offset in xrange(-1, 2):
        if word_index + idx_offset < 0:
            active_word = "<s>"
        elif word_index + idx_offset >= len(sentence):
            active_word = "</s>"
        else:
            active_word = sentence.tokens[word_index + idx_offset].word
        if word_index + idx_offset < 0:
            active_pos = "<S>"
        elif word_index + idx_offset >= len(sentence):
            active_pos = "</S>"
        else:
            active_pos = sentence.tokens[word_index + idx_offset].pos
        maybe_add_feature(feats, feature_indexer, add_to_indexer, tag + ":Word" + repr(idx_offset) + "=" + active_word)
        maybe_add_feature(feats, feature_indexer, add_to_indexer, tag + ":Pos" + repr(idx_offset) + "=" + active_pos)
    # Character n-grams of the current word
    max_ngram_size = 3
    for ngram_size in xrange(1, max_ngram_size+1):
        start_ngram = curr_word[0:min(ngram_size, len(curr_word))]
        maybe_add_feature(feats, feature_indexer, add_to_indexer, tag + ":StartNgram=" + start_ngram)
        end_ngram = curr_word[max(0, len(curr_word) - ngram_size):]
        maybe_add_feature(feats, feature_indexer, add_to_indexer, tag + ":EndNgram=" + end_ngram)
    # Look at a few word shape features
    maybe_add_feature(feats, feature_indexer, add_to_indexer, tag + ":IsCap=" + repr(curr_word[0].isupper()))
    maybe_add_feature(feats, feature_indexer, add_to_indexer, tag + ":IsAllCap=" + repr(curr_word.isupper()))
    maybe_add_feature(feats, feature_indexer, add_to_indexer, tag + ":dashExists=" + repr("-" in curr_word))
    # Compute word shape
    new_word = []
    for i in xrange(0, len(curr_word)):
        if curr_word[i].isupper():
            new_word += "X"
        elif curr_word[i].islower():
            new_word += "x"
        elif curr_word[i].isdigit():
            new_word += "0"
        else:
            new_word += "?"
    maybe_add_feature(feats, feature_indexer, add_to_indexer, tag + ":WordShape=" + repr(new_word))
    return np.asarray(feats, dtype=int)


class CrfNerModel(object):
    def __init__(self, tag_indexer, feature_indexer, feature_weights):
        self.tag_indexer = tag_indexer
        self.feature_indexer = feature_indexer
        self.feature_weights = feature_weights

    # Takes a LabeledSentence object and returns a new copy of that sentence with a set of chunks predicted by
    # the CRF model. See BadNerModel for an example implementation
    def decode(self, sentence):
        feature_cache = [[[] for k in xrange(0, len(self.tag_indexer))] for j in xrange(0, len(sentence))]
        for word_idx in range(0, len(sentence)):
            for tag_idx in range(0, len(self.tag_indexer)):
                feature_cache[word_idx][tag_idx] = extract_emission_features(
                    sentence, word_idx, 
                    self.tag_indexer.get_object(tag_idx), 
                    self.feature_indexer, 
                    add_to_indexer=False)

                
        # Viterbi
        score = np.zeros((len(sentence), len(self.tag_indexer)))
        back_pointers = np.ones((len(sentence), len(self.tag_indexer))) * -1
        sequence_scorer = FeatureBasedSequenceScorer(self.tag_indexer, self.feature_indexer, self.feature_weights)
        for word_idx in xrange(0, len(sentence)):
            if word_idx == 0:
                for tag_idx in xrange(0, len(self.tag_indexer)):
                    tag = self.tag_indexer.get_object(tag_idx)
                    if isI(tag):
                        score[word_idx][tag_idx] = -np.inf
                    else:    
                        score[word_idx][tag_idx] = sequence_scorer.score_init(feature_cache, tag_idx)
            else:
                for curr_tag_idx in xrange(0, len(self.tag_indexer)):
                    score[word_idx][curr_tag_idx] = -np.inf
                    for prev_tag_idx in xrange(0, len(self.tag_indexer)):
                        # TODO : did not prohibit the O-I transition at the last word
                        curr_tag = self.tag_indexer.get_object(curr_tag_idx)
                        prev_tag = self.tag_indexer.get_object(prev_tag_idx)
                        if isO(prev_tag) and isI(curr_tag):
                            continue
                        if isI(curr_tag) and (get_tag_label(curr_tag) != get_tag_label(prev_tag)):
                            continue
                        curr_score = sequence_scorer.score_transition(feature_cache, prev_tag_idx, curr_tag_idx) + \
                                        sequence_scorer.score_emission(feature_cache, curr_tag_idx, word_idx) + score[word_idx-1][prev_tag_idx]
                        if curr_score > score[word_idx][curr_tag_idx]:
                            score[word_idx][curr_tag_idx] = curr_score
                            back_pointers[word_idx][curr_tag_idx] = prev_tag_idx
                            
        
        max_score_idx = score.argmax(axis=1)[-1]
        idx = max_score_idx
        pred_tags = []
        word_idx = len(sentence) - 1
        
        while idx != -1 :
            pred_tags.append(self.tag_indexer.get_object(idx))
            idx = back_pointers[int(word_idx)][int(idx)]
            word_idx -= 1
        pred_tags.reverse()
        
        return LabeledSentence(sentence.tokens, chunks_from_bio_tag_seq(pred_tags))
    
    class Counter(object):
    def __init__(self):
        self.counter = {}

    def __repr__(self):
        return str([str(key) + ": " + str(self.get_count(key)) for key in self.counter.keys()])

    def __len__(self):
        return len(self.counter)

    def keys(self):
        return self.counter.keys()

    def get_count(self, key):
        if self.counter.has_key(key):
            return self.counter[key]
        else:
            return 0

    def increment_count(self, obj, count):
        if self.counter.has_key(obj):
            self.counter[obj] = self.counter[obj] + count
        else:
            self.counter[obj] = count

    def increment_all(self, objs_list, count):
        for obj in objs_list:
            self.increment_count(obj, count)

    def set_count(self, obj, count):
        self.counter[obj] = count

    def add(self, otherCounter):
        for key in otherCounter.counter.keys():
            self.increment_count(key, otherCounter.counter[key])

    # Bad O(n) implementation right now
    def argmax(self):
        best_key = None
        for key in self.counter.keys():
            if best_key is None or self.get_count(key) > self.get_count(best_key):
                best_key = key
        return best_key
    
    # TODO : implementation specific to emission features only, change forward and backward if adding transition features
def compute_log_marginals(sentence, tag_indexer, sentence_feature_cache, feature_weights):
    # find alpha -> forward pass
    log_alpha = np.zeros((len(sentence), len(tag_indexer)))
    for tag_idx in xrange(0, len(tag_indexer)):
        log_alpha[0][tag_idx] = score_indexed_features(sentence_feature_cache[0][tag_idx], feature_weights)
    for word_idx in xrange(1, len(sentence)):
        for tag_idx in xrange(0, len(tag_indexer)):
            log_alpha[word_idx][tag_idx] = -np.inf
            for prev_tag_idx in xrange(0, len(tag_indexer)):
                curr_tag = tag_indexer.get_object(tag_idx)
                prev_tag = tag_indexer.get_object(prev_tag_idx)
                if isI(curr_tag) and get_tag_label(curr_tag) != get_tag_label(prev_tag):
                    continue
                log_alpha[word_idx][tag_idx] = np.logaddexp(
                    log_alpha[word_idx][tag_idx], \
                    log_alpha[word_idx - 1][prev_tag_idx] + \
                    score_indexed_features(sentence_feature_cache[word_idx][tag_idx],
                                           feature_weights))
             
            # log_alpha[word_idx][tag_idx] = scipy.misc.logsumexp(log_alpha[word_idx-1] + score_indexed_features(sentence_feature_cache[word_idx][tag_idx], feature_weights))
    
    # find beta -> backward pass
    log_beta = np.zeros((len(sentence), len(tag_indexer)))
    for word_idx in range(len(sentence)-2, -1, -1):
        for tag_idx in range(0, len(tag_indexer)):
            log_beta[word_idx][tag_idx] = -np.inf
            for next_tag_idx in range(0, len(tag_indexer)):
                curr_tag = tag_indexer.get_object(tag_idx)
                next_tag = tag_indexer.get_object(next_tag_idx)
                if isI(next_tag) and get_tag_label(curr_tag) != get_tag_label(next_tag):
                    continue
                log_beta[word_idx][tag_idx] = np.logaddexp(
                    log_beta[word_idx][tag_idx], \
                    log_beta[word_idx + 1][next_tag_idx] + \
                    score_indexed_features(sentence_feature_cache[word_idx][next_tag_idx], 
                                           feature_weights))
            # tmp = np.apply_along_axis(score_indexed_features, 1, sentence_feature_cache[word_idx], feature_weights)
            # log_beta[word_idx][tag_idx] = scipy.misc.logsumexp(log_beta[word_idx + 1] + tmp)

    # marginal = alpha[word_idx][tag_idx] * beta[word_idx][tag_idx] / Sigma (alpha, beta)
    log_marginal_probs = np.zeros((len(sentence), len(tag_indexer)))
    log_marginal_probs = log_alpha + log_beta
    # denom = np.apply_along_axis(scipy.misc.logsumexp, 1, log_marginal_probs)
    # log_marginal_probs -= denom[:, None]
    for word_idx in range(0, len(sentence)):
        denom = -np.inf
        for tag_idx in range(0, len(tag_indexer)):
            denom = np.logaddexp(denom, log_marginal_probs[word_idx][tag_idx])
        log_marginal_probs[word_idx] -= denom
    return log_marginal_probs



In [ ]:
# Indexes a string feat using feature_indexer and adds it to feats.
# If add_to_indexer is true, that feature is indexed and added even if it is new
# If add_to_indexer is false, unseen features will be discarded
def maybe_add_feature(feats, feature_indexer, add_to_indexer, feat):
    if add_to_indexer:
        feats.append(feature_indexer.get_index(feat))
    else:
        feat_idx = feature_indexer.index_of(feat)
        if feat_idx != -1:
            feats.append(feat_idx)
            
class FeatureBasedSequenceScorer(object):
    def __init__(self, tag_indexer, feature_indexer, feature_weights):
        self.tag_indexer = tag_indexer
        self.feature_indexer = feature_indexer
        self.feature_weights = feature_weights

    def score_init(self, feature_cache, tag_idx):
        return score_indexed_features(feature_cache[0][tag_idx], self.feature_weights)

    def score_transition(self, feature_cache, prev_tag_idx, curr_tag_idx):
        return 0

    def score_emission(self, feature_cache, tag_idx, word_idx):
        return score_indexed_features(feature_cache[word_idx][tag_idx], self.feature_weights)

# Thin wrapper over a sequence of Tokens representing a sentence and an optional set of chunks
# representation NER labels, which are also stored as BIO tags
class LabeledSentence:
    def __init__(self, tokens, chunks=None):
        self.tokens = tokens
        self.chunks = chunks
        if chunks is None:
            self.bio_tags = None
        else:
            self.bio_tags = bio_tags_from_chunks(self.chunks, len(self.tokens))

    def __repr__(self):
        return repr([repr(tok) for tok in self.tokens]) + "\n" + repr([repr(chunk) for chunk in self.chunks])

    def __len__(self):
        return len(self.tokens)

    def get_bio_tags(self):
        return self.bio_tags
    
def score_indexed_features(feats, weights):
    score = 0.0
    # for feat in feats:
    #     score += weights[feat]
    score = np.take(weights, feats).sum()
    return score